In [ ]:
from pystackql import StackQL
import json
import pandas as pd
provider_auth =  {
    "github": {
        "credentialsenvvar": "GITHUBCREDS",
        "type": "basic"
    }
}
stackql = StackQL(auth=provider_auth)

In [ ]:
stackql.executeStmt("REGISTRY PULL github")

In [ ]:
org = "stackql"

In [ ]:
query = "select name, stargazers_count from github.repos.repos where org = '%s' order by stargazers_count desc limit 10" % (org)
stargazers = pd.read_json(stackql.execute(query)).sort_values('stargazers_count', ascending=False)

In [ ]:
stargazers.plot(kind='bar', title='Stargazers', x='name', y='stargazers_count')

In [ ]:
query = "select name from github.repos.repos where org = '%s'" % (org)
repos = pd.read_json(stackql.execute(query))

In [ ]:
# star all repos in an org
for index, row in repos.iterrows():
    repo = row['name']
    print("starring %s" % repo)
    query = """
    EXEC github.activity.starred_repos.star_repo_for_authenticated_user @owner='%s', @repo='%s'
    """ % (org, repo)
    stackql.executeStmt(query)

